In [1]:
# !git clone https://github.com/cg123/mergekit.git
# !cd mergekit && pip install -q -e .

Cloning into 'mergekit'...
remote: Enumerating objects: 1243, done.
remote: Counting objects: 100% (614/614), done.
remote: Compressing objects: 100% (219/219), done.
remote: Total 1243 (delta 511), reused 411 (delta 393), pack-reused 629
Receiving objects: 100% (1243/1243), 341.86 KiB | 5.03 MiB/s, done.
Resolving deltas: 100% (847/847), done.

[notice] A new release of pip is available: 23.3.1 -> 24.0
[notice] To update, run: python -m pip install --upgrade pip


In [2]:
import yaml

yaml_config = """
slices:
  - sources:
      - model: mlabonne/AlphaMonarch-7B
        layer_range: [0, 32]
      - model: bardsai/jaskier-7b-dpo-v5.6
        layer_range: [0, 32]
merge_method: slerp
base_model: mlabonne/AlphaMonarch-7B
parameters:
  t:
    - filter: self_attn
      value: [0, 0.5, 0.3, 0.7, 1]
    - filter: mlp
      value: [1, 0.5, 0.7, 0.3, 0]
    - value: 0.5
dtype: bfloat16

"""

# Save config as yaml file
with open('config.yaml', 'w', encoding="utf-8") as f:
    f.write(yaml_config)

In [3]:
# Merge models
!mergekit-yaml config.yaml merge --copy-tokenizer --allow-crimes --out-shard-size 1B --lazy-unpickle

Fetching 10 files:   0%|                                 | 0/10 [00:00<?, ?it/s]

generation_config.json: 100%|███████████████████| 116/116 [00:00<00:00, 518kB/s]

Fetching 10 files:  20%|█████                    | 2/10 [00:03<00:13,  1.68s/it]

model.safetensors.index.json: 100%|████████| 23.9k/23.9k [00:00<00:00, 58.4MB/s]


tokenizer_config.json: 100%|███████████████| 1.25k/1.25k [00:00<00:00, 10.0MB/s]


special_tokens_map.json: 100%|█████████████████| 624/624 [00:00<00:00, 3.01MB/s]


tokenizer.json: 100%|██████████████████████| 1.80M/1.80M [00:00<00:00, 27.9MB/s]


model-00001-of-00003.safetensors:   0%|             | 0.00/4.94G [00:00<?, ?B/s]

model-00001-of-00003.safetensors:   0%|    | 10.5M/4.94G [00:00<00:57, 86.0MB/s]

model-00001-of-00003.safetensors:   0%|    | 21.0M/4.94G [00:00<00:57, 85.4MB/s]

model-00001-of-00003.safetensors:   1%|     | 41.9M/4.94G [00:00<00:39, 124MB/s]


model-00002-of-00003.safetensors:   0%|             | 0.00/5.00G [00:00<?, ?B/s]



tokenizer

In [6]:
!pip install -qU huggingface_hub

from huggingface_hub import ModelCard, ModelCardData
from jinja2 import Template
username = "CorticalStack"
MODEL_NAME = "crown-clown-7B-slerp"


[notice] A new release of pip is available: 23.3.1 -> 24.0
[notice] To update, run: python -m pip install --upgrade pip


In [7]:
template_text = """
---
license: apache-2.0
tags:
- merge
- mergekit
{%- for model in models %}
- {{ model }}
{%- endfor %}
---

# {{ model_name }}

{{ model_name }} is a Spherical Linear Interpolation (SLERP) merge of the following models using [mergekit](https://github.com/cg123/mergekit):

{%- for model in models %}
* [{{ model }}](https://huggingface.co/{{ model }})
{%- endfor %}

## 🧩 Configuration

```yaml
{{- yaml_config -}}
```
"""

In [8]:
# Create a Jinja template object
jinja_template = Template(template_text.strip())

# Get list of models from config
data = yaml.safe_load(yaml_config)
if "models" in data:
    models = [data["models"][i]["model"] for i in range(len(data["models"])) if "parameters" in data["models"][i]]
elif "parameters" in data:
    models = [data["slices"][0]["sources"][i]["model"] for i in range(len(data["slices"][0]["sources"]))]
elif "slices" in data:
    models = [data["slices"][i]["sources"][0]["model"] for i in range(len(data["slices"]))]
else:
    raise Exception("No models or slices found in yaml config")

# Fill the template
content = jinja_template.render(
    model_name=MODEL_NAME,
    models=models,
    yaml_config=yaml_config,
    username=username,
)

# Save the model card
card = ModelCard(content)
card.save('merge/README.md')

In [11]:
from huggingface_hub import HfApi
HF_TOKEN = ""

# Defined in the secrets tab in Google Colab
api = HfApi(token=HF_TOKEN)

api.create_repo(
    repo_id=f"{username}/{MODEL_NAME}",
    repo_type="model"
)
api.upload_folder(
    repo_id=f"{username}/{MODEL_NAME}",
    folder_path="merge",
)

model-00003-of-00008.safetensors:   0%|          | 0.00/1.97G [00:00<?, ?B/s]

model-00002-of-00008.safetensors:   0%|          | 0.00/1.95G [00:00<?, ?B/s]

model-00001-of-00008.safetensors:   0%|          | 0.00/1.98G [00:00<?, ?B/s]

Upload 9 LFS files:   0%|          | 0/9 [00:00<?, ?it/s]

model-00004-of-00008.safetensors:   0%|          | 0.00/1.98G [00:00<?, ?B/s]

...

model-00006-of-00008.safetensors:   0%|          | 0.00/1.92G [00:00<?, ?B/s]

model-00007-of-00008.safetensors:   0%|          | 0.00/1.95G [00:00<?, ?B/s]

model-00008-of-00008.safetensors:   0%|          | 0.00/789M [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/CorticalStack/crown-clown-7B-slerp/commit/bfe7413f586fe57ce629e04b2cb08b67a8775bc7', commit_message='Upload folder using huggingface_hub', commit_description='', oid='bfe7413f586fe57ce629e04b2cb08b67a8775bc7', pr_url=None, pr_revision=None, pr_num=None)